In [1]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [2]:
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [3]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mp
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models


In [4]:
!pip install kaggle

In [6]:
kaggledir=json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME']=kaggledir['username']
os.environ['KAGGLE_KEY']=kaggledir['key']

In [7]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
100% 2.04G/2.04G [00:14<00:00, 241MB/s]
100% 2.04G/2.04G [00:14<00:00, 155MB/s]


In [8]:
!ls


drive  kaggle.json  plantvillage-dataset.zip  sample_data


In [9]:
with ZipFile('plantvillage-dataset.zip','r') as zf:
  zf.extractall()

In [10]:
print (len(os.listdir("plantvillage dataset/color")))
print (os.listdir("plantvillage dataset/color")[:5])

38
['Tomato___healthy', 'Raspberry___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Tomato___Tomato_mosaic_virus']


In [11]:
base_dir = "plantvillage dataset/color"

In [12]:
# 224 us this most commun to use

image_size = 224
batch_size = 32

In [13]:
imagepath= '/content/plantvillage dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'
print(mp.imread(imagepath))

[[[167 163 196]
  [166 162 195]
  [165 161 194]
  ...
  [185 182 211]
  [183 180 209]
  [181 178 207]]

 [[170 166 199]
  [169 165 198]
  [168 164 197]
  ...
  [186 183 212]
  [184 181 210]
  [183 180 209]]

 [[170 166 199]
  [169 165 198]
  [168 164 197]
  ...
  [188 185 214]
  [186 183 212]
  [185 182 211]]

 ...

 [[100  90 124]
  [102  92 126]
  [104  94 128]
  ...
  [126 110 137]
  [126 110 137]
  [126 110 137]]

 [[101  91 125]
  [102  92 126]
  [103  93 127]
  ...
  [122 106 133]
  [123 107 134]
  [123 107 134]]

 [[104  94 128]
  [103  93 127]
  [102  92 126]
  ...
  [121 105 132]
  [122 106 133]
  [123 107 134]]]


In [14]:
data_gen=ImageDataGenerator(rescale=1./255,
                            validation_split=0.2)
train_gen = data_gen.flow_from_directory(
                base_dir,
                target_size=(image_size , image_size ),
                batch_size=batch_size,
                subset ='training',
                class_mode='categorical'
                )
validation_gen = data_gen.flow_from_directory(
                base_dir,
                target_size=(image_size , image_size ),
                batch_size=batch_size,
                subset ='validation',
                class_mode='categorical'
                )

Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


In [15]:
#relu and softmax are activation funcs
model = models.Sequential()

model.add(layers.Conv2D (32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D (64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(train_gen.num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      47,776,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 38)                  │           9,766 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,805,158 (182.36 MB)

 Trainable params: 47,805,158 (182.36 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history= model.fit(train_gen,
          steps_per_epoch=train_gen.samples // batch_size,
          epochs=5,
          validation_data=validation_gen,
          validation_steps= validation_gen.samples // batch_size)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 600/1358 ━━━━━━━━━━━━━━━━━━━━ 44s 59ms/step - accuracy: 0.4815 - loss: 2.7778

In [ ]:
val_loss, val_acc = model.evaluate(validation_gen, steps=validation_gen.samples // batch_size)


In [ ]:
import matplotlib.pyplot as plt
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

epochs = range(1, len(val_loss) + 1)

plt.figure(figsize=(8, 6))

plt.plot(epochs, val_loss, 'b-', label='Validation Loss')

plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')

plt.title('Validation Loss and Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Value')

plt.legend()

plt.show()



In [ ]:

def load_and_preprocess_image (image_path, target_size=(224, 224)):
  img = Image.open(image_path)
  img=img.resize(target_size)
  img_array = np.array(img)
  # Add batch dimension
  img_array = np.expand_dims (img_array, axis=0)
  #Scale the image values to [0, 1]
  img_array =img_array.astype('float32') / 255.
  return img_array

def predict_image_class (model, image_path, class_indices):
  preprocessed_img = load_and_preprocess_image (image_path)
  predictions = model. predict (preprocessed_img)
  predicted_class_index = np.argmax (predictions, axis=1)[0]
  predicted_class_name = class_indices [predicted_class_index]
  return predicted_class_name

In [ ]:
class_indices = {v: k for k, v in train_gen.class_indices.items()}
json.dump(class_indices, open('class_indices.json', 'w'))

In [ ]:
image_path='/content/plantvillage dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'
#image_path='/content/test_blueberry_healthy.jpg'
#image_path='/content/test_potato_early_blight.jpg'
predicted_class_name = predict_image_class (model, image_path, class_indices)
print("Predicted Class Name:", predicted_class_name)

In [ ]:
model.save('/content/plant_model/my_model.h5')

In [ ]:
model.save('C:\\Users\\Hp\\Desktop\\Plant desease\\model.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')